# 缓存

## 缓存雪崩、缓存击穿和缓存穿透

### 缓存雪崩

什么是缓存雪崩？

当某一时刻出现大面积缓存失效的情况，导致大量的请求直接打在数据库上，导致数据库的压力巨大，在高并发场景下，可能导致数据库宕机。

分析

造成缓存雪崩的关键是同一时间大量缓存key失效，为什么会出现这种情况？使用了相同的过期时间，或者Redis宕机。

解决

1. Reis Sentinel高可用方案，提高容灾能力
2. 使用熔断机制，流量超过一定的阈值时，直接返回系统繁忙
3. 设置key的过期时间，加一个随机值

### 缓存击穿

什么是缓存击穿?

其实跟缓存雪崩有点类似，缓存雪崩是大规模的key失效，而缓存击穿是一个热点的Key，有大并发集中对其进行访问，突然间这个Key失效了，导致大并发全部打在数据库上，导致数据库压力剧增。这种现象就叫做缓存击穿。

分析

关键在于某个热点的key失效了，导致大并发集中打在数据库上。所以要从两个方面解决，第一是否可以考虑热点key不设置过期时间，第二是否可以考虑降低打在数据库上的请求数量。

解决

如果业务允许的话，对于热点的key可以设置永不过期的key。

### 缓存穿透

什么是缓存穿透？

我们使用Redis大部分情况都是通过Key查询对应的值，假如发送的请求传进来的key是不存在Redis中的，那么就查不到缓存，查不到缓存就会去数据库查询。假如有大量这样的请求，这些请求像“穿透”了缓存一样直接打在数据库上，这种现象就叫做缓存穿透。

分析

关键在于在Redis查不到key值，这和缓存击穿有根本的区别，区别在于缓存穿透的情况是传进来的key在Redis中是不存在的。假如有黑客传进大量的不存在的key，那么大量的请求打在数据库上是很致命的问题，所以在日常开发中要对参数做好校验，一些非法的参数，不可能存在的key就直接返回错误提示，要对调用方保持这种“不信任”的心态。

解决方案

把无效的Key存进Redis中。如果Redis查不到数据，数据库也查不到，我们把这个Key值保存进Redis，设置value="null"，当下次再通过这个Key查询时就不需要再查询数据库。这种处理方式肯定是有问题的，假如传进来的这个不存在的Key值每次都是随机的，那存进Redis也没有意义。

# 秒杀

参考文献https://www.zixuephp.net/article-499.html

```python
# encoding: utf8

import time
import threading
from redis import Redis
from redis.exceptions import WatchError

total_goods = 10
GOT_USER_KEY = 'buy_users'
GOT_NUM_KEY = 'got_num'

conn = Redis()


def buy(user_id):
    if int(conn.get(GOT_NUM_KEY)) >= total_goods:
        print('已售罄')
        return

    with conn.pipeline() as p:
        try:
            p.watch(GOT_NUM_KEY)

            p.multi()
            p.hset(GOT_USER_KEY, user_id, int(time.time()))
            p.incr(GOT_NUM_KEY)
            ret = p.execute()
            print('%s 抢购成功, ret: %s' % (user_id, ret))
        except WatchError:
            print('%s 抢购失败，请稍后重试！' % user_id)


def main():
    ts = []
    conn.set(GOT_NUM_KEY, 0)
    conn.delete(GOT_USER_KEY)

    for i in range(100):
        t = threading.Thread(target=buy, args=(i,))
        ts.append(t)

    for t in ts:
        t.start()

    for t in ts:
        t.join()

    print('done')


if __name__ == '__main__':
    main()
    
```

# References

* *https://www.iteye.com/blog/carlosfu-2248185*
* *https://baijiahao.baidu.com/s?id=1619572269435584821&wfr=spider&for=pc*
* *https://www.zixuephp.net/article-499.html*